### Problem Statement

The BBC points out that global meat consumption has [increased rapidly over the last 50 years](https://www.bbc.com/news/health-47057341). The article states that the richer the country, the greater the meat consumption. If we were to identify trends and patterns in meat consumption that are associated with changes in GDP in [GDP data collected from Wikipedia](https://en.wikipedia.org/wiki/List_of_countries_by_past_and_projected_GDP_(nominal)_per_capita), then we can provide evidence to the idea that GDP growth does indeed influcence meat consumption. [Meat production has a large environmental impact](https://www.greenpeace.org.uk/news/why-meat-is-bad-for-the-environment/), which includes deforestation to build industrial meat farms, billions of tons of carbon dioxide being released into the atmosphere, feed sourcing, and manure processing, to name a few. Therefore, countries that consume the most amount of meat should do their part to reduce the environmental impact of meat consumption.

### Additional Data Links

- [Meat consumption data from OECD](https://data.oecd.org/agroutput/meat-consumption.htm)

### Import Libraries

In [20]:
import pandas as pd
import plotly.express as px

### Import Data

The yearly GDP data was collected from Wikipedia and consists of estimates from the UN. It has already ben adjusted by a per capita basis. Note that we could use `pd.read_html` to automatically import the data from Wikipedia and then use indexing to select which tables we would want, which in our case are the tables that give us the UN estimates. 

However, from personal experience, `pd.read_html` takes about a minute to import the data. I felt that this was a bit of a long wait, so I collected the data from the tables after using `pd.read_html` once, and then saved the data to a `.csv`. Then I uploaded the data into the forked repository for this class. So now I can quickly import the `.csv` file into this notebook. 

Depending on the data, this might not be the best option. For example, for data that is updated frequently, importing a static data file might not be the best option. However, for our project, this data has not been updated since 2015, and even if it was updated, it would probably only be done on a once-a-year basis since this is YEARLY GDP by capita data. Therefore, for all intents and purposes, importing the data from a static `.csv` file seems like a good option for this project.

In [94]:
# OECD Meat Consumption Data
meat_consumption_df = pd.read_csv(
    filepath_or_buffer="https://raw.githubusercontent.com/peterphung2043/CUNY_DATA_608/master/Final%20Project/data/world_meat_consumption.csv"
)

# Wikipedia GDP Data
gdp_per_cap_df = pd.read_csv(
    filepath_or_buffer="https://raw.githubusercontent.com/peterphung2043/CUNY_DATA_608/master/Final%20Project/data/wikipedia_gdp_data.csv",
    usecols=range(1, 48)
).melt(id_vars = 'Country (or dependent territory)').rename(columns = {'variable': 'Year', "value": "GDP Per Capita"}).astype({'Year': int})

#2 and 3 letter country codes
country_codes_df = pd.read_html(
    io = "https://www.iban.com/country-codes"
)[0]

#World GDP per Capita Time Series Data
pd.read_csv(
    "https://api.worldbank.org/v2/en/indicator/NY.GDP.PCAP.CD?downloadformat=csv"
)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xc0 in position 10: invalid start byte

# Data Pre-Processing

### Display Imported Dataframes

We display the first 5 dataframes as shown below.

In [22]:
display(meat_consumption_df.head())
display(gdp_per_cap_df.head())
display(country_codes_df.head())

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,MEATCONSUMP,BEEF,KG_CAP,A,1990,0.000,NaN
1,AUS,MEATCONSUMP,BEEF,KG_CAP,A,1991,27.942,NaN
2,AUS,MEATCONSUMP,BEEF,KG_CAP,A,1992,26.405,NaN
3,AUS,MEATCONSUMP,BEEF,KG_CAP,A,1993,26.370,NaN
4,AUS,MEATCONSUMP,BEEF,KG_CAP,A,1994,25.662,NaN


,Country (or dependent territory),Year,GDP Per Capita
0,Afghanistan,1970,157.0
1,Albania,1970,1053.0
2,Algeria,1970,354.0
3,Andorra,1970,4097.0
4,Angola,1970,604.0


,Country,Alpha-2 code,Alpha-3 code,Numeric
0,Afghanistan,AF,AFG,4
1,Åland Islands,AX,ALA,248
2,Albania,AL,ALB,8
3,Algeria,DZ,DZA,12
4,American Samoa,AS,ASM,16


Note that `Value` in the `meat_consumption_df` dataframe represents either:
- The amount of kilograms of meat consumed (see the `MEASURE` variable in the same dataframe)
- The amount of meat consumed in thousands of tonnes (not shown in the output above, but is represented as `THND_TONNE` in the `MEASURE` variable)

The `SUBJECT` variable in the `meat_consumption_df` dataframe represents the meat type that is consumed. The different meat types are:
- `BEEF`
- `PIG`
- `POULTRY`
- `SHEEP`

### Merging Meat Consumption with GDP Per Capita Data

Notice that the `meat_consumption_df` each row of `LOCATION` contains a 3 letter abbreviation of the country that the observation is representing. The `Country (or dependent territory)` column in the `gdp_per_cap_df` shows the full name of the country. In order to merge the meat consumption data with the GDP per capita data, we need to replace the `LOCATION` column with the full name of the country, which we will do using the `country_codes_df`.

In [23]:
meat_consumption_modified = pd.merge(meat_consumption_df, country_codes_df, left_on='LOCATION', right_on='Alpha-3 code')[['SUBJECT', 'MEASURE', 'TIME', 'Value', 'Country']]
meat_consumption_modified.head()

,SUBJECT,MEASURE,TIME,Value,Country
0,BEEF,KG_CAP,1990,0.000,Australia
1,BEEF,KG_CAP,1991,27.942,Australia
2,BEEF,KG_CAP,1992,26.405,Australia
3,BEEF,KG_CAP,1993,26.370,Australia
4,BEEF,KG_CAP,1994,25.662,Australia


Note that the `meat_consumption_modified` dataframe does not contain 3 `LOCATION`s that were in the original `meat_consumption_df`. These 3 `LOCATIONS` are: 

- `OECD`: [An intergovernmental organisation with 38 member countries](https://en.wikipedia.org/wiki/OECD)
- `BRICS`: An acronym for 5 leading economies: Brazil, Russia, India, China, and South Africa
- `WLD`: The entire world

Now that we have the full country name, we can merge the meat consumption data with the GDP per capita data.

Before we do that however, since we have meat consumption data for the entire world, we might as well insert the GDP Per Capita data for the entire world inside the `gdp_per_cap_df` dataframe.

### Meat Consumption Data Cleanup

Some of the columns are not needed from the meat consumption dataset. These include:
- `INDICATOR`: Let's the user know which agricultural sector the observation represents, since OECD provides more agricultural data. Since the imported data is all meat consumption however, this column is useless.
- `FREQUENCY`: Represents the time frequency of the observation. Since all of the data was collected at an annual frequency, there is only one value that this column can take, which also makes this column useless.

The amount of meat consumed can be represented in two ways based on the `MEASURE` variable:

- `KG_CAP`: The amount of kilograms of meat eaten per capita
- `THND_TONNE`: Annual consumption in thousands of tonnes

Both of these represent the same information (amount of meat consumed) but in different units of measurement. So for this project, only the meat consumed in `KG_CAP` is considered. Therefore, we can get rid of the `MEASURE` variable, since all of the values in `VALUE` represent just the `KG_CAP`.

In [24]:
meat_and_gdp_df = pd.merge(meat_consumption_modified, gdp_per_cap_df, left_on=['Country', 'TIME'], right_on = ['Country (or dependent territory)', 'Year']).drop(columns = ['Country (or dependent territory)', 'TIME'])
meat_and_gdp_df = meat_and_gdp_df.query('MEASURE == \'KG_CAP\'').drop(columns = ['MEASURE'])

In [25]:
meat_and_gdp_df.query("Country == \'Australia\' & SUBJECT == \'BEEF\'")

,SUBJECT,Value,Country,Year,GDP Per Capita
0,BEEF,0.000,Australia,1990,18940.0
8,BEEF,27.942,Australia,1991,19015.0
16,BEEF,26.405,Australia,1992,18591.0
24,BEEF,26.370,Australia,1993,17888.0
32,BEEF,25.662,Australia,1994,20205.0
40,BEEF,25.526,Australia,1995,21634.0
48,BEEF,27.453,Australia,1996,23777.0
56,BEEF,29.000,Australia,1997,23600.0
64,BEEF,26.765,Australia,1998,20826.0
72,BEEF,28.095,Australia,1999,22547.0


Ultimately, we are trying to see if there is a correlation between GDP Per Capita and Meat Consumption Per Capita. What I propose is that we plot:

- `GDP Per Capita` on the left y-axis
- `Value` on the right y-axis
- `Year` on the x-axis

We can also display the correlation between the `GDP Per Capita` and the `Value` for each `SUBJECT`

### Dash App Layout

In [45]:
def add_figure_layers_by_subject(fig: go.Figure, df: pd.DataFrame) -> go.Figure:
    """Adds figure layers by SUBJECT from the `meat_and_gdp_df`.

    Args:
        fig (go.Figure): 
            Plotly figure object

    Returns:
        go.Figure: 
            Plotly figure object with meat consumption data plotted by SUBJECT in different layers
    """
    for meat_type in df['SUBJECT'].unique():
        subject_df = df.query("SUBJECT == \'{}\'".format(meat_type))
        fig.add_trace(
            go.Scattergl(
                x = subject_df['Year'],
                y = subject_df['Value'],
                name = 'KG {} Consumed Per Capita'.format(meat_type)
            ),
            secondary_y = False
        )
    return fig

def generate_correlation_df(
        df: pd.DataFrame, 
        variable_1: str = 'Value', 
        variable_2: str = 'GDP Per Capita', 
        grouping_variables_list: list[str] = ['Country', 'SUBJECT']):
    """Calculates correlation between `variable_1` and `variable_2` after grouping by `grouping_variables_list` for `df`

    Args:
        df (pd.DataFrame): 
            The dataframe to perform the groupby and calculate the correlation.
        variable_1 (str, optional): 
            One of the correlation variables. Defaults to 'Value'.
        variable_2 (str, optional): 
            One of the correlation variables. Defaults to 'GDP Per Capita'.
        grouping_variables_list (list[str], optional): 
            The `by` parameter in the `pandas.groupby` function uses this variable. Defaults to ['Country', 'SUBJECT'].
    """
    corr_df = df.groupby(['Country', 'SUBJECT'])[['Value', 'GDP Per Capita']].corr().drop('GDP Per Capita', level = 2).reset_index()
    return corr_df.drop(columns = ['level_2', 'Value']).rename(columns = {'GDP Per Capita': 'Correlation between {} and {}'.format(variable_1, variable_2)})

In [69]:
# Run this app with `python app.py` and
# visit http://127.0.0.1:8050/ in your web browser.

from dash import Dash, dcc, html, dash_table
from dash.dependencies import Input, Output
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# Array consisting of different meat types
subject_array = meat_and_gdp_df['SUBJECT'].unique().tolist()
subject_array.append('ALL')

external_stylesheets = [
    {
        "href": (
            "https://fonts.googleapis.com/css2?"
            "family=Lato:wght@400;700&display=swap"
        ),
        "rel": "stylesheet",
    },
]
app = Dash(__name__, external_stylesheets=external_stylesheets)
app.title = "GDP and Meat Consumption Per Capita Dash App"

app.layout = html.Div(
    children=[
        html.Div(
            children = [
                html.H1(children='GDP and Meat Consumption Per Capita Dash App',
                        className = "header-title"),
                html.P(
                    children = (
                        "Exploration of GDP Per Capita collected from Wikipedia"
                        " and meat consumption data collected from OECD."
                    ),
                    className = "header-description"
                ),
                html.Hr()
            ],
            className = "header"
        )
    ,
    dcc.RadioItems(options=subject_array, value='BEEF', id='subject_selection'),    
            
    dcc.Dropdown(id='country_selection', options=[
    {'label': x, 'value': x} for x in meat_and_gdp_df['Country'].unique()
    ],
            value = 'Australia'),

    dcc.Graph(
        id='example-graph'
    )
    ]
)

@app.callback(Output('example-graph', 'figure'),
              [Input('subject_selection', 'value'),
               Input('country_selection', 'value')])

def update_figure(subject_selection, country_selection):

    df = meat_and_gdp_df.query("Country == \'{}\'".format(country_selection))

    # Create figure with secondary y-axis
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    if subject_selection == 'ALL':
        fig = add_figure_layers_by_subject(fig, df)
    else:
        fig.add_trace(
            go.Scattergl(x = df.query("SUBJECT == \'{}\'".format(subject_selection))['Year'], 
                        y = df.query("SUBJECT == \'{}\'".format(subject_selection))['Value'],
                        name = "KG Consumed Per Capita",
                        line = dict(color='firebrick', width=4, dash='dot')),
            secondary_y=False,
        )

    fig.add_trace(
        go.Scatter(x = df['Year'], 
                   y = df['GDP Per Capita'],
                   name = "GDP Per Capita",
                   line=dict(color='royalblue', width=4, dash='dot')),
        secondary_y=True,
    )

    # Add figure title
    fig.update_layout(
        title_text="KG Consumed and GDP Per Capita for {} in {}".format(subject_selection, country_selection)
    )

    # Set x-axis title
    fig.update_xaxes(title_text="Year")

    # Set y-axes titles
    fig.update_yaxes(title_text="KG Consumed Per Capita", secondary_y=False)
    fig.update_yaxes(title_text="GDP Per Capita", secondary_y=True)

    return fig

if __name__ == '__main__':
    app.run_server(debug = True, port = 8000, use_reloader = False)

Dash is running on http://127.0.0.1:8000/

 * Serving Flask app '__main__'
 * Debug mode: on


In [93]:
corr_df = generate_correlation_df(meat_and_gdp_df)

### Generate Bar Chart of Correlations for Each Country